# Import Stuff

In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import model_from_json

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
np.random.seed(1212)

# Prepare Data for Training

In [6]:
df_train=pd.read_csv('train_final.csv', index_col=False)
labels = df_train[['784']]

In [7]:
df_train.drop(df_train.columns[[784]], axis=1, inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,100,208,178,178,...,0,0,0,0,0,0,0,0,0,0
1,0,0,136,255,180,191,185,77,14,0,...,0,0,0,0,0,0,0,0,0,0
2,0,123,193,178,178,178,178,178,178,178,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,36,77,138,178,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,165,178,178,178,113,0,...,18,0,0,0,0,0,0,0,0,0


In [8]:
labels = np.array(labels)

In [9]:
y = to_categorical(labels, num_classes=16)

In [10]:
l = []
for i in range(df_train.shape[0]):
    l.append(np.array(df_train[i:i+1]).reshape(1, 28, 28))

In [11]:
l2 = []
for i in l:
    l2.append(i.reshape(28, 28, 1))
X = np.array(l2)

# CNN Model Training

In [12]:
model = Sequential()

model.add(Conv2D(8, (3, 3), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(16, activation='softmax'))

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
print(X_train.shape)

(37120, 28, 28, 1)


In [16]:
model.fit(X_train, y_train, epochs=10, batch_size=200, shuffle=True, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
186/186 [==============================] - 15s 80ms/step - loss: 2.8095 - accuracy: 0.3963 - val_loss: 1.0359 - val_accuracy: 0.6834
Epoch 2/10
186/186 [==============================] - 17s 92ms/step - loss: 0.9195 - accuracy: 0.7163 - val_loss: 0.5940 - val_accuracy: 0.8111
Epoch 3/10
186/186 [==============================] - 18s 99ms/step - loss: 0.6290 - accuracy: 0.7970 - val_loss: 0.4418 - val_accuracy: 0.8534
Epoch 4/10
186/186 [==============================] - 15s 83ms/step - loss: 0.4790 - accuracy: 0.8398 - val_loss: 0.3553 - val_accuracy: 0.8802
Epoch 5/10
186/186 [==============================] - 12s 65ms/step - loss: 0.4003 - accuracy: 0.8638 - val_loss: 0.3027 - val_accuracy: 0.8972
Epoch 6/10
186/186 [==============================] - 11s 57ms/step - loss: 0.3388 - accuracy: 0.8804 - val_loss: 0.2732 - val_accuracy: 0.9059
Epoch 7/10
186/186 [==============================] - 11s 58ms/step - loss: 0.3025 - accuracy: 0.8908 - val_loss: 0.2418 - val_accuracy:

# Save to JSON

In [18]:
model_json = model.to_json()

with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model_final.h5")